# LSTM model to perform sentiment analysis on a 50k movie reviews - Fellowship AI challenge

in the following notebook, we performed sentiment analysis on 50k movie reviews, using LSTM (Long Short Term Memory) network, which is a variant of RNN-recurrent neural netwok that is capable of learning long-term dependencies. 

The built of the algorithm consists of an embedding layer, an LSTM layer and a dense layer. a Dropout mechanism was introduced in-between layers of LSTM to avoid overfitting.

first let's import and read our dataset 

In [127]:
import pandas as pd
df = pd.read_csv("/Users/oumai/Desktop/NLP/IMDB.csv")

In [128]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


we will then turn the categorical data in the 'sentiment' column to numerical one, representing 0 for positive, and 1 for negative. and then we will turn the two columns into arrays.

In [129]:
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

review_data = df['review'].to_numpy()
sentiment_label = df['sentiment'].to_numpy()

In [130]:
review_data

array(["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to

now, we will break down the long sentences into tokens using Keras Tokenizer. the "num_word=10000" tells the program to show the most used 10000 words in the array. the "fit_on_texts" command creates a connection between words and their assigned numbers.

In [131]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000)

tokenizer.fit_on_texts(review_data)

now we will replace the words with their assigned numbers

In [132]:
numbered_words = tokenizer.texts_to_sequences(review_data)

the next step is to ensure that all sentence are of the same length. for this, we will use pre-padding since it is better for the model we will be using for the analysis, which is the LSTM.

In [133]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_words = pad_sequences(numbered_words, maxlen=200)

Now, into our sentiment analysis using LSTM. to start, we will first import the libraries we will be needing, and then add an embedding layer to convert our words to embedding vectors 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

vocab_size = 10000
embedding_vector_length = 32
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200))

next we define a sequential() model to embed the layers of LSTM

In [ ]:
model = Sequential()

now we added the Dropout mechanism  

In [ ]:
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))

since our output will be between 0 and 1, we will use sigmoid activation with 1-unit

In [ ]:
model.add(Dense(1, activation='sigmoid'))

the 'adam' optemizer used has a faster computation time, and require fewer parameters for tuning. and it will also provide us with information for each epoch that runs. the 'binary_crossentroy' function is the loss function, that will be optemized and minimized by the 'adam' optemizer. and last, we will print a summary of our model.

In [136]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 200, 32)           320000    
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 200, 32)          0         
 lDropout1D)                                                     
                                                                 
 lstm_5 (LSTM)               (None, 50)                16600     
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 51        
                                                                 
Total params: 336,651
Trainable params: 336,651
Non-trainable params: 0
________________________________________________

time to fit the model to our data to run for 5 epochs and a batch size of 32

In [137]:
history = model.fit(padded_words, sentiment_label, validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
1250/1250 [==============================] - 642s 507ms/step - loss: 0.4149 - accuracy: 0.8075 - val_loss: 0.2901 - val_accuracy: 0.8871
Epoch 2/5
1250/1250 [==============================] - 452s 362ms/step - loss: 0.2813 - accuracy: 0.8879 - val_loss: 0.3146 - val_accuracy: 0.8776
Epoch 3/5
1250/1250 [==============================] - 585s 468ms/step - loss: 0.2439 - accuracy: 0.9044 - val_loss: 0.2999 - val_accuracy: 0.8850
Epoch 4/5
1250/1250 [==============================] - 660s 528ms/step - loss: 0.2248 - accuracy: 0.9130 - val_loss: 0.3026 - val_accuracy: 0.8811
Epoch 5/5
1250/1250 [==============================] - 876s 701ms/step - loss: 0.2079 - accuracy: 0.9196 - val_loss: 0.2942 - val_accuracy: 0.8852


the sentiment analysis model yielded an accuracy of 91.9% on the trainning set, and an accuracy of 88% on the testing set